In [2]:
import pandas as pd
import os
from datetime import datetime
import string
import random
import re

def generate_matrix_code():
    """Generate a matrix code starting with 'SCO' followed by 4 random uppercase letters."""
    return "SCO" + ''.join(random.choices(string.ascii_uppercase, k=4))

def px_escape(text):
    """Escape quotes in strings for PX format."""
    if isinstance(text, str):
        return text.replace('"', '""')
    return text

def suggest_columns(df):
    """
    Suggest stub_cols, heading_cols, and value_col based on DataFrame content.
    Returns a dictionary with suggestions.
    """
    suggestions = {
        'value_col': None,
        'stub_cols': [],
        'heading_cols': [],
        'notes': []
    }
    
    # Identify potential value column (numeric, likely a measurement)
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    for col in numeric_cols:
        if df[col].notna().any():
            # Prioritize columns with "rating", "energy", or similar terms
            if any(keyword in col.lower() for keyword in ['rating', 'energy', 'score', 'value']):
                suggestions['value_col'] = col
                suggestions['notes'].append(f"Suggested '{col}' as value_col (numeric, measurement-related).")
                break
            # Fallback: numeric with decimals or moderate cardinality
            elif df[col].dropna().apply(lambda x: x % 1 != 0).any() or (10 < df[col].nunique() < 500):
                suggestions['value_col'] = col
                suggestions['notes'].append(f"Suggested '{col}' as value_col (numeric with decimals or moderate cardinality).")
                break
    
    if not suggestions['value_col']:
        suggestions['notes'].append("No clear value_col found. Please specify manually.")
    
    # Remaining columns for stub and heading
    non_value_cols = [col for col in df.columns if col != suggestions['value_col']]
    categorical_cols = df[non_value_cols].select_dtypes(include=['object', 'category']).columns
    
    # Suggest stub_cols (time, geographic, or moderate cardinality)
    for col in categorical_cols:
        unique_count = df[col].nunique()
        if unique_count > 1:
            if any(keyword in col.lower() for keyword in ['date', 'year', 'quarter', 'month']):
                suggestions['stub_cols'].append(col)
                suggestions['notes'].append(f"Suggested '{col}' for stub_cols (time-related).")
            elif any(keyword in col.lower() for keyword in ['town', 'city', 'region', 'area', 'postcode']):
                suggestions['stub_cols'].append(col)
                suggestions['notes'].append(f"Suggested '{col}' for stub_cols (geographic).")
            elif 20 <= unique_count <= 100:
                suggestions['stub_cols'].append(col)
                suggestions['notes'].append(f"Suggested '{col}' for stub_cols (moderate cardinality: {unique_count} unique values).")
    
    # Suggest heading_cols (low cardinality, e.g., ratings, types)
    for col in categorical_cols:
        if col not in suggestions['stub_cols']:
            unique_count = df[col].nunique()
            if 1 < unique_count <= 20:
                suggestions['heading_cols'].append(col)
                suggestions['notes'].append(f"Suggested '{col}' for heading_cols (low cardinality: {unique_count} unique values).")
    
    # Limit stub_cols to 3 to avoid excessive combinations
    if len(suggestions['stub_cols']) > 3:
        suggestions['stub_cols'] = suggestions['stub_cols'][:3]
        suggestions['notes'].append("Limited stub_cols to first 3 to avoid excessive combinations.")
    
    # Ensure at least one stub_col
    if not suggestions['stub_cols'] and suggestions['heading_cols']:
        suggestions['stub_cols'].append(suggestions['heading_cols'].pop(0))
        suggestions['notes'].append("Moved one heading_col to stub_cols as no stub_cols were identified.")
    
    return suggestions

def tidy_to_pxstat(
    input_file,
    output_file=None,
    stub_cols=None,
    heading_cols=None,
    title=None,
    subject_area=None,
    matrix_code=None,
    decimals=None,
    value_col="Value",
    source="Scottish Government",
    agg_method="mean"
):
    """
    Convert Tidy format CSV to monolingual PxStat format.
    
    Parameters:
    -----------
    input_file : str
        Path to input CSV file.
    output_file : str, optional
        Path to output PX file (default: input_name + ".px").
    stub_cols : list, optional
        Columns to use as stub dimensions (rows).
    heading_cols : list, optional
        Columns to use as heading dimensions (columns).
    title : str, optional
        Title for the PX dataset.
    subject_area : str, optional
        Subject area for the dataset.
    matrix_code : str, optional
        Matrix code for the dataset (default: generated as SCO+4 random letters).
    decimals : int, optional
        Number of decimals to use.
    value_col : str, optional
        Column name containing the values (default: "Value").
    source : str, optional
        Data source (default: "Scottish Government").
    agg_method : str, optional
        Aggregation method for duplicates ("sum" or "mean", default: "mean").
    """
    print(f"Loading data from {input_file}...")
    try:
        # Load CSV with optimized dtypes
        df = pd.read_csv(input_file, low_memory=False)
        
        # Convert categorical columns to 'category' dtype
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].astype('category')
        
        # Validate required columns
        if value_col not in df.columns:
            raise ValueError(f"Missing required column: {value_col}")
        
        # Use provided or default dimensions
        if stub_cols is None or heading_cols is None:
            suggestions = suggest_columns(df)
            stub_cols = stub_cols or suggestions['stub_cols']
            heading_cols = heading_cols or suggestions['heading_cols']
            print(f"Using stub_cols: {stub_cols}")
            print(f"Using heading_cols: {heading_cols}")
        
        # Validate dimensions
        group_cols = stub_cols + heading_cols
        if not group_cols:
            raise ValueError("No valid dimension columns specified or detected.")
        for col in group_cols:
            if col not in df.columns:
                raise ValueError(f"Dimension column '{col}' not found in data.")
        
        # Simplify to needed columns
        df_simple = df[group_cols + [value_col]].copy()
        
        # Make values numeric
        df_simple[value_col] = pd.to_numeric(df_simple[value_col], errors="coerce", downcast='float')
        
        # Fix duplicates by grouping with observed=True
        df_simple = df_simple.groupby(group_cols, as_index=False, observed=True).agg({value_col: agg_method})
        
        # Use existing combinations (avoid full_index to prevent memory explosion)
        indexed_df = df_simple.set_index(group_cols)
        
        # Generate data values
        data_values = [
            '".."' if pd.isna(v) else str(round(v, decimals or 3))
            for v in indexed_df[value_col]
        ]
        
        # Metadata
        creation_date = datetime.today().strftime("%Y%m%d %H:%M")
        if title is None:
            title = f"Data from {os.path.basename(input_file)}"
        if subject_area is None:
            subject_area = "Statistics"
        if matrix_code is None:
            matrix_code = generate_matrix_code()
        if decimals is None:
            max_decimals = max(
                [len(str(x).split('.')[-1]) if '.' in str(x) else 0 for x in df_simple[value_col].dropna()]
            ) if not df_simple[value_col].dropna().empty else 0
            decimals = min(max_decimals, 6)
        
        units = px_escape(df['Units'].iloc[0] if 'Units' in df.columns and not df['Units'].isna().all() else 'Count')
        header = f"""CHARSET="UTF-16";
AXIS-VERSION="2013";
CREATION-DATE="{creation_date}";
MATRIX="{matrix_code}";
DECIMALS={decimals};
SUBJECT-AREA="{px_escape(subject_area)}";
SUBJECT-CODE="{matrix_code[:4] if len(matrix_code) >= 4 else matrix_code}";
CONTENTS="{px_escape(title)}";
TITLE="{px_escape(title)} - by {', '.join(group_cols)}";
UNITS="{units}";
STUB="{','.join(f'"{px_escape(col)}"' for col in stub_cols)}";
HEADING="{','.join(f'"{px_escape(col)}"' for col in heading_cols)}";
SOURCE="{px_escape(source)}";
"""
        
        # VALUES and CODES blocks
        dim_vals = {col: sorted(df_simple[col].dropna().unique()) for col in group_cols}
        def px_values_and_codes(name, values):
            quoted_vals = ",".join(f'"{px_escape(str(v))}"' for v in values)
            quoted_codes = ",".join(f'"{i+1:02d}"' for i in range(len(values)))
            return f'VALUES("{name}")={quoted_vals};\nCODES("{name}")={quoted_codes};\n'
        
        meta_parts = "".join(px_values_and_codes(col, dim_vals[col]) for col in group_cols)
        
        # Make output filename
        if output_file is None:
            output_file = os.path.splitext(input_file)[0] + ".px"
        
        # Write to file
        print(f"Writing {len(data_values)} data points to {output_file}...")
        with open(output_file, "w", encoding="utf-16") as f:
            f.write(header)
            f.write(meta_parts)
            f.write("DATA=\n")
            f.write(" ".join(data_values) + "\n")
            f.write(";")
        
        print(f"✅ PX file saved as: {output_file}")
        return output_file
    
    except Exception as e:
        print(f"❌ Error processing file: {str(e)}")
        raise

# Step 1: Declare input file
INPUT_FILE = "Fire - Type of Incident.csv"

# Step 2: Analyze file and suggest columns
print(f"Analyzing {INPUT_FILE}...")
df_preview = pd.read_csv(INPUT_FILE, nrows=1000)  # Load a sample for analysis
suggestions = suggest_columns(df_preview)

print("\nSuggested Columns:")
print(f"Value Column: {suggestions['value_col']}")
print(f"Stub Columns: {suggestions['stub_cols']}")
print(f"Heading Columns: {suggestions['heading_cols']}")
print("\nNotes:")
for note in suggestions['notes']:
    print(f"- {note}")

# Step 3: Configuration template (modify as needed)
CONFIG = {
    'input_file': INPUT_FILE,
    'output_file': "Fire - Type of Incident.px",
    'value_col': suggestions['value_col'],  # Override if needed
    'stub_cols': suggestions['stub_cols'],  # Custom override
    'heading_cols': suggestions['heading_cols'],  # Custom override
    'title': "Fire - Type of Incident",
    'subject_area': "Incidents",
    'matrix_code': generate_matrix_code(),  # e.g., SCOABCD
    'decimals': 3,
    'source': "Scottish Fire and Rescue Service",
    'agg_method': "mean"
}

# Step 4: Run the conversion
print("\nRunning conversion with the following configuration:")
for key, value in CONFIG.items():
    print(f"{key}: {value}")

tidy_to_pxstat(**CONFIG)

Analyzing Fire - Type of Incident.csv...

Suggested Columns:
Value Column: Value
Stub Columns: ['DateCode']
Heading Columns: ['FeatureCode', 'FeatureName', 'Accident Status', 'Type of Fire']

Notes:
- Suggested 'Value' as value_col (numeric, measurement-related).
- Suggested 'DateCode' for stub_cols (time-related).
- Suggested 'FeatureCode' for heading_cols (low cardinality: 3 unique values).
- Suggested 'FeatureName' for heading_cols (low cardinality: 3 unique values).
- Suggested 'Accident Status' for heading_cols (low cardinality: 3 unique values).
- Suggested 'Type of Fire' for heading_cols (low cardinality: 8 unique values).

Running conversion with the following configuration:
input_file: Fire - Type of Incident.csv
output_file: Fire - Type of Incident.px
value_col: Value
stub_cols: ['DateCode']
heading_cols: ['FeatureCode', 'FeatureName', 'Accident Status', 'Type of Fire']
title: Fire - Type of Incident
subject_area: Incidents
matrix_code: SCOSRHE
decimals: 3
source: Scottish Fi

'Fire - Type of Incident.px'